In [6]:
import json
import pandas as pd

In [3]:
dicio_conversao = json.load(open('cruzamento_infosiga-lentidao.json'))

In [7]:
lentidao = (
    pd.read_csv("dados/lentidaotrechos2022.csv", encoding="latin", sep = ";")
    .replace({"corredor": {"Av": "Avenida", ",": "", "/": " ", "\(": " ", "\)": " ", "\.": " "}}, regex = True)
)

In [16]:
([av for av in list(dicio_conversao.values()) if av != None])

['João De Luca Ver',
 'Ataliba Leonel Avenida  Gal ',
 'Ibirapuera Avenida',
 'Abraão de Morais Avenida Prof Imig',
 'Carlos Caldeira Filho Avenida',
 'Caetano Alvares Avenida  Eng ',
 'Corifeu de A Marques Avenida',
 'Celso Garcia Avenida',
 'Morumbi Avenida',
 'Amaro Avenida Sto DEC SA',
 'Liberdade  Vergueiro Avenida',
 'Inajar de Souza Avenida',
 'João Dias Avenida',
 'Radial Leste - DEC BR',
 'Nove de Julho Avenida',
 'Fernando Vieira de Mello Túnel Reboucas ',
 'Luis Antonio Avenida Brig',
 'Raimundo Pereira de Magalhães - Norte',
 'Salim Farah Maluf Avenida Tatuapé Pte',
 'Atlantica AV',
 'Francisco Morato Avenida Prof',
 'Guarapiranga Avenida',
 'Luiz Ignácio de Anhaia Mello Avenida Prof',
 'Vinte Três R Berta M Guimarães',
 'Estado Avenida do - DEC CT',
 'Bandeirantes Avenida dos',
 'Interlagos Avenida I',
 'Washington Luis Avenida',
 'Aricanduva Avenida Elev Pt',
 'Itapecerica Est de',
 'F1 - Teotonio Vilela Avenida Sen']

In [28]:
lentidao.query(f"corredor == 'Bandeirantes Avenida dos'").descricao.unique()

array(['De CABO VERDE até IRAUNA',
       'De ANTONIO DE MACEDO SOARES até AMARO Vd',
       'De ANTONIO DE MACEDO SOARES até RIBEIRO DO VALE',
       'De JOAO CARLOS MALLET até RIBEIRO DO VALE',
       'De ANTONIO DE MACEDO SOARES até ISABEL',
       'De BANDEIRANTES Vd até EDUARDO SACCAB',
       'De 220 m antes de EDUARDO SACCAB até NOVA YORK',
       'De CONSTANTINO DE SOUSA até TRIUNFO',
       'De 773 m antes de MIRUNA até ALIOMAR BALEEIRO Término',
       'De CABO VERDE até MARACATINS',
       'De MARGINAL PINHEIROS até JULIO DINIZ',
       'De MARGINAL PINHEIROS até ALVORADA Pça Carananduba',
       'De JABAQUARA Vd até JOAO CARLOS MALLET',
       'De 776 m antes de MIRUNA até ARAPUA Vd',
       'De 250 m antes de JOAO CARLOS MALLET até JOAO JULIAO DA COSTA AGUIAR Viaduto',
       'De MARGINAL PINHEIROS até FUNCHAL',
       'De CABO VERDE até BANDEIRANTES Vd',
       'De ALIOMAR BALEEIRO Início até 190 m depois de ZACARIAS DE GOIS',
       'De ANTONIO DE MACEDO SOARES até MARGI

In [26]:
lentidao.query(f"corredor in {list(dicio_conversao.values())}").groupby("corredor").agg({"descricao": "nunique"})

,descricao
corredor,
Abraão de Morais Avenida Prof Imig,23
Amaro Avenida Sto DEC SA,17
Aricanduva Avenida Elev Pt,162
Ataliba Leonel Avenida Gal,6
Atlantica AV,37
Bandeirantes Avenida dos,428
Caetano Alvares Avenida Eng,2
Carlos Caldeira Filho Avenida,31
Celso Garcia Avenida,14


In [5]:
dicio_conversao.values()

dict_values(['João De Luca Ver', None, None, 'Ataliba Leonel Avenida  Gal ', 'Ibirapuera Avenida', None, None, None, 'Abraão de Morais Avenida Prof Imig', None, None, None, None, None, 'Carlos Caldeira Filho Avenida', 'Caetano Alvares Avenida  Eng ', None, None, 'Corifeu de A Marques Avenida', None, 'Celso Garcia Avenida', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'Morumbi Avenida', 'Amaro Avenida Sto DEC SA', None, 'Liberdade  Vergueiro Avenida', 'Inajar de Souza Avenida', None, 'João Dias Avenida', 'Radial Leste - DEC BR', None, None, None, None, None, None, None, 'Nove de Julho Avenida', 'Fernando Vieira de Mello Túnel Reboucas ', None, None, None, 'Luis Antonio Avenida Brig', 'Raimundo Pereira de Magalhães - Norte', None, None, 'Salim Farah Maluf Avenida Tatuapé Pte', 'Atlantica AV', None, 'Francisco Morato Avenida Prof', 'Guarapiranga Avenida', 'Luiz Ignácio de Anhaia Mello Avenida Prof', None, 'Vinte Três R Berta M Guimarães',